![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Configuring Egeria Servers Lab

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information about data and technology.  This information is called metadata.

In this hands-on lab you will learn how to configure the metadata servers used by [Coco Pharmaceuticals](https://opengovernance.odpi.org/coco-pharmaceuticals/).

## The scenario

<img src="https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/gary-geeke.png" style="float:left">

Coco Pharmaceuticals is going through a major business transformation that requires them to drastically reduce their cycle times, collaborate laterally across the different parts of the business and react quickly to the changing needs of their customers. (See [this link](https://opengovernance.odpi.org/coco-pharmaceuticals/) for the background to this transformation).

Part of the changes needed to the IT systems that support the business is the roll out of a distributed open metadata and governance capability that is provided by Egeria.

[Gary Geeke](https://opengovernance.odpi.org/coco-pharmaceuticals/personas/gary-geeke.html) is the IT Infrastructure leader at Coco Pharmaceuticals.

In this hands-on lab Gary is configuring the servers that support this open ecosystem.  These servers are collectively called Open Metadata and Governance (OMAG) Servers.

Gary's userId is `garygeeke`.

In [ ]:
import requests

adminUserId     = "garygeeke"

He needs to define the OMAG servers for Coco Pharmaceuticals.

In [ ]:
organizationName = "Coco Pharmaceuticals"

## Open Metadata and Governance (OMAG) management landscape

At the heart of an open metadata and governance landscape are the servers that store and exchange metadata in a peer-to-peer exchange called the
[open metadata repository cohort](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/cohort-member.html).
These servers are collectively called **cohort members**.  There are three types of cohort member that Gary needs to consider:

* A [Metadata Server](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/metadata-server.html) that uses
  a native Egeria repository to store open metadata.  There should be at least one of these servers in a cohort. It used to support
  either a community of users that are using the Egeria functions directly or to fill in any gaps in the metadata support provided by the
  third party tools that are connected to the cohort.
  
* A [Metadata Access Point](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/metadata-access-point.html) that 
  has no metadata repository of its own and uses federated queries to retrieve and store metadata in the other repositories connected to the cohort.
  
* A [Repository Proxy](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/repository-proxy.html)  that connects
  in a thrid party metadata server.

Gary has decided to deploy a separate cohort member server for each part of the organization that owns 
[assets](https://egeria.odpi.org/open-metadata-implementation/access-services/docs/concepts/assets/).
You can think of each of these servers as supporting a community of users within Coco Pharmaceuticals.  The servers are as follows:

* cocoMDS1 - Data Lake Operations - a **metadata server** used to manage the data in the data lake.
* cocoMDS2 - Governance - a **metadata server** used by all of the governance teams to operate the governance programs.
* cocoMDS3 - Research - a **metadata server** used by the research teams who are developing new treatments.
* cocoMDS4 - Data Lake Users - a **metadata access point** used by general business users and the executive team to access data
  from the data lake.
* cocoMDS5 - Business Systems - a **repository proxy** used to connect to the existing ETL tool that manages data movement amongst the
  business systems.  It has a metadata record of the operational business systems such as procurements, sales, human resources and
  finance and the movement of data between them.  This tool is also loading data from the business systems into the data lake. Its metadata
  is critical for providing lineage for the data used to run the business.
* cocoMDS6 - Manufacturing - a **metadata server** used by the supplies warehouse, manufacturing and distribution teams.
* cocoMDSx - Development - a **metadata server** used by the software development teams building new IT capablity.
* cocoEDGEi - Manufacturing sensors edge node servers (many of them) - these **metadata servers** catalog the collected sensor data.

In addition, Coco Pharmaceuticals needs additional servers to support Egeria's user interface and automated metadata processing:

* cocoView1 - a [View Server](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/view-server.html) 
  that runs the services for the user interface.           
* exchangeDL01 - an [Integration Daemon](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/integration-daemon.html) 
  server that supports the automatic exchange of metadata with third party technologies.          
* governDL01 - an [Engine Host](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/engine-host.html)
  server that runs governance functions that monitor, validate, correct and enrich metadata for use by all of the technologies in the connected
  open metadata ecosystem.
* cocoOLS1 - an Open Lineage [Governance Server](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/governance-server-types.html) that manages a historical warehouse of lineage information.

These servers will each be configured in later parts of this hands-on lab, but first there are decisons to be made about the platform that the servers will run on and how they will be connected together.

### Open Metadata and Governance (OMAG) Server Platforms

Coco Pharmaceuticals' servers must be hosted on at least one OMAG Server Platform.
This is a single executable (application) that can be started from the command line or a script or as part of a 
pre-built container environment `kubernetes`.

If you are running this notebook as part of an Egeria hands on lab then the server platforms you need are already started.  Run the following command to check that the  platforms are running.


In [ ]:
%run common/environment-check.ipynb

----
If one of the platforms is not running, follow [this link to set up and run the platform](https://egeria.odpi.org/open-metadata-resources/open-metadata-labs/).  Once the platforms are running you are ready to proceed.

----
Most of the servers are supporting a pretty stable environment and can share an OMAG Server Platform because the workload they are supporting is predicable.
The data lake however requires a lot of active governance and is evolving rapidly.  
To isolate this churn, Gary chooses to put all of the metadata and governance servers for the data lake on to their own platform.
The development team requested that their infrastructure is completely separate from the operational systems,
so they are given their own server platform.
Finally each of the edge servers will run their own OMAG Server Platform to support their own metadata server.

Figure 1 shows which servers will sit in each platform.  The cohort members are shown in white,
governance servers in orange and the view server (that supports the UI) is in green.

![Figure 1](images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms


The sensor edge node servers used to monitor the warehouse operation and manufacturing process each have their own platform and are not yet included in this notebook.

### Open Metadata Repository Cohorts

A metadata server, metadata access point and repository proxy can become a member of none, one or many cohorts.
Once a server has joined a cohort it can exchange metadata with the other members of that cohort.
So the cohorts define scopes of sharing.

Gary decides to begin with three open metadata repository cohorts:

* **cocoCohort** - The production cohort contains all of the servers that are used to run, coordinate and govern the business.
* **devCohort** - The development cohort where the development teams are building and testing new capablity.  Much of their metadata describes the software components under construction and the governance of the software development lifecycle.
* **iotCohort** - The IoT cohort used to manage the sensors and robots in the manufacturing systems.  The metadata produced by the sensors and robots is only of interest to the manufactuing and governance team.

Figure 2 shows which servers belong to each cohort.

![Figure 2](images/coco-pharmaceuticals-systems-cohorts.png)
> **Figure 2:** Membership of Coco Pharmaceuticals' cohorts

Below are the names of the three cohorts.

In [ ]:
cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"

At the heart of each cohort is an event topic.  By default, Egeria uses [Apache Kafka](https://kafka.apache.org/) topics.
The servers that wil ljoin a cohort will need to be configured with the host name and port where Kafka is running.
The command below pulls the value from an environment variable called `eventBusURLroot` with a default value of
`localhost:9092`.  It is used in all of the server configuration documents to connect it to Kafka.

In [ ]:
eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')

jsonContentHeader = {'content-type':'application/json'}
eventBusBody      = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
    },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}

## Access services

[The Open Metadata Access Services (OMAS)](https://egeria.odpi.org/open-metadata-implementation/access-services/) provide domain-specific services for data tools, engines and platforms to integrate with open metadata. These are the different types of access service.

In [ ]:

getAccessServices(cocoMDS1PlatformName, cocoMDS1PlatformURL)


The table below shows which access services are needed by each server.


| Access Service       | cocoMDS1 | cocoMDS2 | cocoMDS3 | cocoMDS4 | cocoMDS5 | cocoMDS6 | cocoMDSx | cocoEDGE*i* |
| :------------------- | :------: | :------: | :------: | :------: | :------: | :------: | :------: | :---------: |
| asset-catalog        |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| asset-consumer       |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| asset-owner          |   Yes    |   Yes    |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| community-profile    |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| glossary-view        |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| data-science         |   No     |   No     |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| subject-area         |   No     |   Yes    |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| governance-program   |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| data-privacy         |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| security-officer     |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| asset-lineage        |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| discovery-engine     |   Yes    |   No     |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| governance-engine    |   Yes    |   Yes    |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| asset-manager        |   Yes    |   No     |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| data-engine          |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     Yes     |
| data-manager         |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   Yes    |     Yes     |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| it-infrastructure    |   No     |   Yes    |   No     |   No     |   No     |   Yes    |   Yes    |     No      |
| project-management   |   No     |   Yes    |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| software-developer   |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| devops               |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| digital-architecture |   Yes    |   Yes    |   No     |   No     |   No     |   No     |   Yes    |     No      |
| design-model         |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |

## Egeria Server Configuration Overview

Open metadata servers are configured using REST API calls to an OMAG Server Platform.  Each call either defines a default value or configures a service that must run within the server when it is started.

As each configuration call is made, the OMAG Server Platform builds up a [configuration document](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/configuration-document.html) with the values passed.  When the configuration is finished, the configuration document will have all of the information needed to start the server.

The configuration document will then be deployed with the OMAG Server Platform that is to host the server.  When a request is made to this OMAG Server Platform to start the server, it reads the configuration document and initializes the server with the appropriate services.

## Configuration Set Up

A server can be configured by any OMAG Server Platform - it does not have to be the same platform where the server will run.  For this hands on lab we will use the development team's OMAG Server Platform to create the servers' configuration documents and then deploy them to the platforms where they will run.

In [ ]:
adminPlatformURL = devPlatformURL

The URLs for the configuration REST APIs have a common structure and begin with the following root:

In [ ]:
adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

Many of Coco Pharmaceuticals' metadata servers need a local repository to store metadata about the data and processing occuring in the data lake.

Egeria includes two types of repositories natively. One is an **in-memory repository** that stores metadata in hash maps. It is useful for demos and testing because a restart of the server results in an empty metadata repository. However, if you need metadata to persist from one run of the server to the next, you should use the **local graph repository**.

The choice of local repository is made by specifying the local repository mode. The variables below show the two options. The `metadataRepositoryType` identfies which one is going to be used in the configuration.

In [ ]:
inMemoryRepositoryOption = "in-memory-repository"
graphRepositoryOption    = "local-graph-repository"

# Pick up which repo type to use from environment if set, otherwise default to inmemory 
metadataRepositoryType   = os.environ.get('repositoryType', inMemoryRepositoryOption)


Egeria supports instance based security.  These checks can be customized through an
[Open Metadata Security Connector](https://egeria.odpi.org/open-metadata-implementation/common-services/metadata-security/).
Coco Pharaceuticals have written their own connector to support the specific rules of their industry.
The Connection definition below tells a server how to load this connector.  It needs to be included in each server's configuration document.

In [ ]:
serverSecurityConnectionBody = {
    "class": "Connection",
    "connectorType": {
        "class": "ConnectorType",
        "connectorProviderClassName": "org.odpi.openmetadata.metadatasecurity.samples.CocoPharmaServerSecurityProvider"
    }
}

Finally, to ensure that a caller can not request too much metadata in a single request, it is possible to set a maximum page size for requests that return a list of items.  The maximum page size puts a limit on the number of items that can be requested.  The variable below defines the value that will be added to the configuration document for each server.

In [ ]:
maxPageSize = '600'

## Configuring cocoMDS1 - Data Lake Operations metadata server

This section configures the `cocoMDS1` server.  The server name is passed on every configuration call to identify which configuration document to update with the new configuration.  The configuration document is created automatically on first use.

In [ ]:
mdrServerName          = "cocoMDS1"
mdrServerUserId        = "cocoMDS1npa"
mdrServerPassword      = "cocoMDS1passw0rd"
mdrServerPlatform      = dataLakePlatformURL
metadataCollectionName = "Data Lake Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can"]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)

accessServiceOptions["DefaultZones"] = [ "quarantine" ]
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'discovery-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'digital-architecture', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-lineage', accessServiceOptions)

print("\nDone.")

----

## Configuring cocoMDS2 - Governance metadata server

This section configures the `cocoMDS2` server.  This server is configured in a similar way to cocoMDS1 except that is has different Open Metadata Access Services (OMASs) enabled and it joins all of the cohorts.

The code below covers the basic set up of the server properties, security, event bus and local repository.

In [ ]:
mdrServerName          = "cocoMDS2"
mdrServerUserId        = "cocoMDS2npa"
mdrServerPassword      = "cocoMDS2passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Governance Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
# Note: cohort membership is configured for all of the cohorts here
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, devCohort)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, iotCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-engine', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-program', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-privacy', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'digital-architecture', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'security-officer', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-lineage', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'it-infrastructure', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'project-management', {})

print("\nDone.")

----

## Configuring cocoMDS3 - Research

Server cocoMDS3 is used by the research teams who are developing new treatments.
These teams are working with their own assets as well as assets coming from the data lake.
So they have their own repository and connector to the core cohort to access all of the
operational metadata.

This is one of the big changes brought by Coco Pharaceuticals' business transformation.
In their old business model, the research teams were completely separate from the operational
part of the organization.  Now they need to be an active member of the day to day running of
the organization, supporting the development of personalized medicines and their use in
treating patients.

In [ ]:
mdrServerName          = "cocoMDS3"
mdrServerUserId        = "cocoMDS3npa"
mdrServerPassword      = "cocoMDS3passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Research Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": ["personal-files", "clinical-trials", "research", "data-lake", "trash-can"]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)

accessServiceOptions["DefaultZones"] = [ "personal-files" ]
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-science', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'discovery-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'project-management', accessServiceOptions)

print("\nDone.")

----
## Configuring cocoMDS4 - Data Lake Users

Server cocoMDS4 used by general business users and the executive team to access data from the data lake.
It does not have a repository of its own.  Instead it issues federated queries to the other repositories in the `cocoCohort`.

In [ ]:
mdrServerName          = "cocoMDS4"
mdrServerUserId        = "cocoMDS4npa"
mdrServerPassword      = "cocoMDS4passw0rd"
mdrServerPlatform      = dataLakePlatformURL
metadataCollectionName = "Data Lake Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
# Note: no metadata repository or collection configuration here
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": [ "data-lake" ]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-science', accessServiceOptions)

print("\nDone.")

----

## Configuring cocoMDS5 - Business Systems

Server cocoMDS5 is a repository proxy to an ETL tool called `iisCore01`.  This ETL tool is well established in Coco Pharmaceuticals and has a built-in metadata repository that contains information about their operational business systems such as procurement, sales, human resources and finance.

This ETL tool has its own user interface and services so the OMASs are not enabled.

In [ ]:
mdrServerName          = "cocoMDS5"
mdrServerUserId        = "cocoMDS5npa"
mdrServerPassword      = "cocoMDS5passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Business Systems Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureRepositoryProxyDetails(adminPlatformURL, adminUserId, mdrServerName, "org.odpi.openmetadata.adapters.repositoryservices.readonly.repositoryconnector.ReadOnlyOMRSRepositoryConnectorProvider")
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)
# Note: no access service configuration here
# Still need to add startup Archive 

print("\nDone.")

----

## Configuring cocoMDS6 - Manufacturing

Server cocoMDS6 is the repository server used by the warehouse, manufacturing and distribution teams.  It supports the systems for this part of the organization and acts as a hub for monitoring the IoT environment.

In [ ]:
mdrServerName          = "cocoMDS6"
mdrServerUserId        = "cocoMDS6npa"
mdrServerPassword      = "cocoMDS6passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Manufacturing Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, iotCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": [ "manufacturing" ],
    "DefaultZones" : [ "manufacturing"]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-science', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'discovery-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'it-infrastructure', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'project-management', accessServiceOptions)

print("\nDone.")

----

## Configuring cocoMDSx - Development

Server cocoMDSx is used by the development teams building new IT capablity.  It will hold all of the software component assets and servers used for development and devOps.  The development teams have their own OMAG Server Platform and cohort called 'devCohort'.

In [ ]:
mdrServerName          = "cocoMDSx"
mdrServerUserId        = "cocoMDSxnpa"
mdrServerPassword      = "cocoMDSxpassw0rd"
mdrServerPlatform      = devPlatformURL
metadataCollectionName = "Development Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, devCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": [ "sdlc", "quarantine", "clinical-trials", "research", "data-lake", "trash-can" ],
    "DefaultZones": [ "sdlc" , "quarantine"]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-science', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'discovery-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'it-infrastructure', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'project-management', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'software-developer', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'devops', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'digital-architecture', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'design-model', accessServiceOptions)

print("\nDone.")

----
## Configuring the exchangeDL01 Integration Daemon

The **exchangeDL01** integration daemon server supports the automatic exchange of metadata with third party technologies.
It runs [integration connectors](https://egeria.odpi.org/open-metadata-implementation/governance-servers/integration-daemon-services/docs/integration-connector.html)
that each connect to a particular third party technology to exchange metadata.

Egeria offers the following Open Metadata Integration Services (OMIS), or integration services for short.  These integration services provide specialist services for an integration connector.  The command below lists the different types of integration services.

In [ ]:

getIntegrationServices(exchangeDL01PlatformName, exchangeDL01PlatformURL)


An integration connector depends on a single integration service.
Gary plans to use two integration connectors supplied by Egeria:

The **DataFilesMonitorIntegrationConnector** maintains a DataFile asset for each file in the directory (or any subdirectory).
When a new file is created, a new DataFile asset is created.  If a file is modified, the lastModified property
of the corresponding DataFile asset is updated.  When a file is deleted, its corresponding DataFile asset is also deleted.

The **DataFolderMonitorIntegrationConnector** maintains a DataFolder asset for the directory.  The files and directories
underneath it are assumed to be elements/records in the DataFolder asset and so each time there is a change to the
files and directories under the monitored directory, it results in an update to the lastModified property
of the corresponding DataFolder asset.

They will be used to automatically catalog data files provided by the different partner hospitals and move them from the
landing area to the data lake once the cataloguing is complete.

Figure 3 shows the integration daemon with its two connectors.
It uses cocoMDS1 to store and retrieve metadata, since that is where the assets for the data lake are catalogued.

![Figure 3](images/integration-daemon.png)
> **Figure 3:** exchangeDL01 with its partner metadata server


### Configuring the server

The commands below configure the integration daemon with the Files Integrator OMIS and the two connectors.  

In [ ]:
daemonServerName        = "exchangeDL01"
daemonServerPlatform    = dataLakePlatformURL
daemonServerUserId      = "exchangeDL01npa"
daemonServerPassword    = "exchangeDL01passw0rd"

mdrServerName           = "cocoMDS1"
mdrServerPlatform       = dataLakePlatformURL

OakDeneConnectorName         = "OakDeneLandingAreaFilesMonitor"
OakDeneConnectorUserId       = "onboardDL01npa"
OakDeneConnectorSourceName   = "HospitalLandingArea"
OakDeneConnectorFolder       = fileSystemRoot + '/landing-area/hospitals/oak-dene/clinical-trials/drop-foot'
OakDeneConnectorConnection   =  { 
                                      "class" : "Connection",
                                      "connectorType" : 
                                      {
                                           "class" : "ConnectorType",
                                           "connectorProviderClassName" : "org.odpi.openmetadata.adapters.connectors.integration.basicfiles.DataFilesMonitorIntegrationProvider"           
                                      },
                                      "endpoint" :
                                      {
                                           "class" : "Endpoint",
                                           "address" : OakDeneConnectorFolder
                                      }
                                }

OldMarketConnectorName         = "OldMarketLandingAreaFilesMonitor"
OldMarketConnectorUserId       = "onboardDL01npa"
OldMarketConnectorSourceName   = "HospitalLandingArea"
OldMarketConnectorFolder       = fileSystemRoot + '/landing-area/hospitals/old-market/clinical-trials/drop-foot'
OldMarketConnectorConnection   =  { 
                                      "class" : "Connection",
                                      "connectorType" : 
                                      {
                                           "class" : "ConnectorType",
                                           "connectorProviderClassName" : "org.odpi.openmetadata.adapters.connectors.integration.basicfiles.DataFilesMonitorIntegrationProvider"           
                                      },
                                      "endpoint" :
                                      {
                                           "class" : "Endpoint",
                                           "address" : OldMarketConnectorFolder
                                      }
                                 }

folderConnectorName       = "DropFootClinicalTrialResultsFolderMonitor"
folderConnectorUserId     = "monitorDL01npa"
folderConnectorSourceName = "DropFootClinicalTrialResults"
folderConnectorFolder     = fileSystemRoot + '/data-lake/research/clinical-trials/drop-foot/weekly-measurements'
folderConnectorConnection =  { 
                                  "class" : "Connection",
                                  "connectorType" : 
                                  {
                                       "class" : "ConnectorType",
                                       "connectorProviderClassName" : "org.odpi.openmetadata.adapters.connectors.integration.basicfiles.DataFolderMonitorIntegrationProvider"           
                                  },
                                  "endpoint" :
                                  {
                                       "class" : "Endpoint",
                                       "address" : folderConnectorFolder
                                  }
                              }

print("Configuring " + daemonServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, daemonServerName, daemonServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, daemonServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, daemonServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, daemonServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, daemonServerName, daemonServerUserId)
configurePassword(adminPlatformURL, adminUserId, daemonServerName, daemonServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, daemonServerName, serverSecurityConnectionBody)
configureDefaultAuditLog(adminPlatformURL, adminUserId, daemonServerName)

print("\nConfiguring " + daemonServerName + " integration connectors ...")

connectorConfigs = [ 
    {
        "class"                       : "IntegrationConnectorConfig",
        "connectorName"               : OakDeneConnectorName,
        "connectorUserId"             : OakDeneConnectorUserId,
        "connection"                  : OakDeneConnectorConnection,
        "metadataSourceQualifiedName" : OakDeneConnectorSourceName,
        "refreshTimeInterval"         : 10,
        "usesBlockingCalls"           : "false"
    },
    {
        "class"                       : "IntegrationConnectorConfig",
        "connectorName"               : OldMarketConnectorName,
        "connectorUserId"             : OldMarketConnectorUserId,
        "connection"                  : OldMarketConnectorConnection,
        "metadataSourceQualifiedName" : OldMarketConnectorSourceName,
        "refreshTimeInterval"         : 10,
        "usesBlockingCalls"           : "false"
    },
    {
        "class"                       : "IntegrationConnectorConfig",
        "connectorName"               : folderConnectorName,
        "connectorUserId"             : folderConnectorUserId,
        "connection"                  : folderConnectorConnection,
        "metadataSourceQualifiedName" : folderConnectorSourceName,
        "refreshTimeInterval"         : 10,
        "usesBlockingCalls"           : "false"
  }]

configureIntegrationService(adminPlatformURL, adminUserId, daemonServerName, mdrServerName, mdrServerPlatform, "files-integrator", {}, connectorConfigs)

print ("\nDone.")

----
## Configuring the monitorDev01 Integration Daemon

The **monitorDev01** is another integration daemon server supporting the development team.  It is cataloguing assets in preparation for deployment into production.  It has no integration services active initially since they are set up in the labs and demos that use it.


In [ ]:
daemonServerName        = "monitorDev01"
daemonServerPlatform    = devPlatformURL
daemonServerUserId      = "erinoverview"
daemonServerPassword    = "erinoverviewpassw0rd"

mdrServerName           = "cocoMDSx"
mdrServerPlatform       = devPlatformURL

print("Configuring " + daemonServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, daemonServerName, daemonServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, daemonServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, daemonServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, daemonServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, daemonServerName, daemonServerUserId)
configurePassword(adminPlatformURL, adminUserId, daemonServerName, daemonServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, daemonServerName, serverSecurityConnectionBody)
configureDefaultAuditLog(adminPlatformURL, adminUserId, daemonServerName)

print ("\nDone.")

----
## Configuring governDL01 Governance Engine Hosting Server

The Engine Host OMAG server is a special kind of governance server that hosts one or more governance engines.

A governance engine is a set of specialized services that perform specific functions to manage the digital landscape and the metadata that describes it.  

### Automated metadata discovery

One example of a type of governance engine is a discovery engine. The discovery engine runs discovery services.  Discovery services analyze the content of a real-world artifact or resource.  For example, a discovery service may open up a data set and assess the quality of the data inside.

The result of a discovery service's analysis is stored in a metadata server as a discovery analysis report that is chained off of the asset's definition.  This report can be retrieved either through the engine host server's API or through the metadata server's APIs, specifically the Discovery Engine OMAS and the Asset Owner OMAS.

The interfaces used by discovery services are defined in the [Open Discovery Framework (ODF)](https://egeria.odpi.org/open-metadata-implementation/frameworks/open-discovery-framework/).  This framework enables new implementations of discovery services to be deployed to the discovery engines.



### Automated governance

Another type of governance engine is a governance action engine.  The governance action engine runs governance action services.
Governance action services monitor the asset metadata and verify that it is set up correctly, determin how to fix anomolies, errors and ommisions,
make the necessary changes and provision real-world artifacts and resources beased on the resulting metadata.



### Understanding the engine services

Coco Pharmaceuticals runs one engine host server for its data lake.  It is called `governDL01` and it runs on the data lake platform.
Within the engine host server there are engine services.  Each engine service supports a specific type of governance engine.
The command below shows you the different types of engine services


In [ ]:

getEngineServices(governDL01PlatformName, governDL01PlatformURL)


----
The governDL01 server is the Engine Host server that runs governance functions that monitor, validate, correct and enrich metadata for use by all of the technologies in the connected open metadata ecosystem.

The **Asset Analysis** Open Metadata Engine Service (OMES) is responsible for running discovery engines from the
[Open Discovery Framework (ODF)](https://egeria.odpi.org/open-metadata-implementation/frameworks/open-discovery-framework/docs/).
Coco Pharmaceuticals has two discovery engines:

* **AssetDiscovery** - extracts metadata about different types of assets on request.
* **AssetQuality** - assesses the quality of the content of assets on request.

The **Governance Action** Open Metadata Engine Service (OMES) is responsible for running governance action engines from the
[Governance Action Framework (GAF)](https://egeria.odpi.org/open-metadata-implementation/frameworks/governance-action-framework/).
Coco Pharmaceuticals has one governance action engine:

* **AssetGovernance** - monitors for new assets in the landing areas, automatically curates them and provisions them in the data lake.

Figure 4 shows the integration daemon with its two connectors.
It uses cocoMDS1 to store and retrieve metadata, since that is where the assets for the data lake are catalogued.

![Figure 4](images/engine-host.png)
> **Figure 4:** Metadata servers for governDL01

### Configuring the server

The commands below configure the engine host server with the Asset Analysis OMES and Governance Action OMES.  
The definitions of the named governance engines and their services are retrieved from the `cocoMDS1` metadata server through its Governance Engine OMAS.


In [ ]:
engineServerName        = "governDL01"
engineServerPlatform    = dataLakePlatformURL
engineServerUserId      = "governDL01npa"
engineServerPassword    = "governDL01passw0rd"

engineServerMDRName     = "cocoMDS2"
engingServerMDRPlatform = corePlatformURL

mdrServerName           = "cocoMDS1"
mdrServerPlatform       = dataLakePlatformURL

print("Configuring " + engineServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, engineServerName, engineServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, engineServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, engineServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, engineServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, engineServerName, engineServerUserId)
configurePassword(adminPlatformURL, adminUserId, engineServerName, engineServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, engineServerName, serverSecurityConnectionBody)
configureDefaultAuditLog(adminPlatformURL, adminUserId, engineServerName)

print("\nConfiguring " + engineServerName + " engines ...")

configureEngineDefinitionServices(adminPlatformURL, adminUserId, engineServerName, engineServerMDRName, engingServerMDRPlatform)

discoveryEngines = [ 
    {
        "class"               : "EngineConfig",
        "engineQualifiedName" : "AssetDiscovery",
        "engineUserId"        : "findItDL01npa"
    }, 
    {
        "class"               : "EngineConfig",
        "engineQualifiedName" : "AssetQuality",
        "engineUserId"        : "findItDL01npa"
    }]

governanceActionEngines = [ 
    {
        "class"               : "EngineConfig",
        "engineQualifiedName" : "AssetGovernance",
        "engineUserId"        : "findItDL01npa"
    }]

configureGovernanceEngineService(adminPlatformURL, adminUserId, engineServerName, mdrServerName, mdrServerPlatform, "asset-analysis", discoveryEngines)
configureGovernanceEngineService(adminPlatformURL, adminUserId, engineServerName, mdrServerName, mdrServerPlatform, "governance-action", governanceActionEngines)

print ("\nDone.")

----

# Configuring the cocoOLS1  - Open Lineage Governance Server

The open lineage server **cocoOLS1** is a dedicated Governance Server that manages a historical warehouse of lineage information.
The Open Lineage Services provides a historic reporting warehouse for lineage. It listens to events that are sent out by the Asset Lineage OMAS, and stores lineage data in a graph database. This lineage can then be queried through the Open Lineage Services Client and by its REST API, for example by a lineage GUI.

![Figure 5](images/open-lineage-server.png)
> **Figure 5:** Open Lineage Governance Server and related services

## Configuring the server

The admin commands below will configure new server with name `cocoOLS1` on the Data Lake Platform.

The server role is configured via `OpenLineageConfig` configuration document. Complete instructions can be found on [Open Lineage Services configuration](https://egeria.odpi.org/open-metadata-implementation/adapters/open-connectors/governance-daemon-connectors/open-lineage-connectors/open-lineage-janus-connector/) page.

For this lab we use following:

- `accessServiceConfig`- Lineage events are received from the output topic connector of Asset Lineage OMAS. Also in some cases when needed, Open Lineage Server will use Asset Lineage OMAS REST client to fetch metadata needed to build detailed graph context. 
- `lineageGraphConnection` - Default configuration for the lineage graph store connector. The default graph store implementation is using `JanusGraph` database, `BerkeleyDB` as local storage and `Lucene` as search index. For more details about the connector see [Lineage Janus Connector](https://egeria.odpi.org/open-metadata-implementation/adapters/open-connectors/governance-daemon-connectors/open-lineage-connectors/open-lineage-janus-connector/)



In [ ]:
lineageServerName        = "cocoOLS1"
lineageServerPlatform    = dataLakePlatformURL

mdrServerName            = "cocoMDS1"
mdrServerUserId          = "cocoMDS1npa"
mdrServerPassword        = "secret"
mdrServerPlatform        = dataLakePlatformURL


In [ ]:
# OLS Server configuration common function
#
#  adminPlatformURL - url of the platform where admin-services are running
#  adminUserId - user with admin access
#  lineageServerName - name of the new server initializing Open Lineage Services
#  metadataPlatformURL - url of the platform hosting the metadata repository server
#  metadataServerName - name of the server where metadata repository is located and asset-lineage omas is running
#  metadataServerUserName - user name with access asset-lineage omas
#  metadataServerUserPassword - password for the user with access to asset-lineage omas


def configureOpenLineageService(adminPlatformURL, adminUserId, lineageServerName, metadataPlatformURL, metadataServerName, metadataServerUserName, metadataServerUserPassword):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the Open Lineage Services server " + lineageServerName + " ...")
    jsonContentHeader = {'content-type':'application/json'}
    requestBody =  {
       "class": "OpenLineageConfig",
       "openLineageDescription": "Open Lineage Service is used for the storage and querying of lineage",
       "lineageGraphConnection": {
         "class": "Connection",
         "displayName": "Lineage Graph Connection",
         "description": "Used for storing lineage in the Open Metadata format",
         "connectorType": {
           "class": "ConnectorType",
           "connectorProviderClassName": "org.odpi.openmetadata.openconnectors.governancedaemonconnectors.openlineageconnectors.janusconnector.graph.LineageGraphConnectorProvider"
         },
         "configurationProperties": {
           "gremlin.graph": "org.janusgraph.core.JanusGraphFactory",
           "storage.backend": "berkeleyje",
           "storage.directory": "data/servers/"+ lineageServerName + "/lineage-repository/berkeley",
           "index.search.backend": "lucene",
           "index.search.directory": "data/servers/"+ lineageServerName + "/lineage-repository/searchindex"
         }
       },
       "accessServiceConfig": {
         "serverName": metadataServerName,
         "serverPlatformUrlRoot": metadataPlatformURL,
         "user": metadataServerUserName,
         "password": metadataServerUserPassword
       },
       "backgroundJobs": [
         {
           "jobName": "LineageGraphJob",
           "jobInterval": 120,
           "jobEnabled": "false"
         }, 
         {
           "jobName": "AssetLineageUpdateJob",
           "jobInterval": 120,
           "jobEnabled": "false",
           "jobDefaultValue": "2021-12-03T10:15:30"
         }
       ]
    }
    print(requestBody)
    url = adminCommandURLRoot + lineageServerName + '/open-lineage/configuration'
    postAndPrintResult(url, json=requestBody, headers=jsonContentHeader)

This server type also depends on standard sub-systems like event bus and audit logging. 

To configure follow the command sequence below. Once it is finished successfully it will print out 'Done.'

In [ ]:
print("Configuring " + lineageServerName + " Open Lineage Services (OLS) ...")

configureEventBus(adminPlatformURL, adminUserId, lineageServerName, eventBusBody)
configureDefaultAuditLog(adminPlatformURL, adminUserId, lineageServerName)
configureOpenLineageService(adminPlatformURL, adminUserId, lineageServerName, mdrServerPlatform, mdrServerName, mdrServerUserId, mdrServerPassword)

print ("\nDone.")

----

# Configuring the View Server and View Services

Egeria's UI allows Coco Pharmaceutical's employees to understand more
about their metadata environment.
This UI uses special services, called view services, that run in an Egeria View Server.

In [ ]:

getViewServices(cocoView1PlatformName, cocoView1PlatformURL)


This is an initial version of an example to configure the view services.
Since this area is still in development the configuration is likely to change, and so all of the
functions are in this section of the notebook rather than consolidated with our common functions.

The new UI is deployed in the k8s environment

The tenant (`coco` in this case) must be explicitly provided in the URL, as must navigation to the login page
For example if the UI is on port 18091,login at https://localhost:18091/coco/login

Further docs will be added in future releases. Please use http://slack.lfai.foundation to get further help

# Deploying server configuration

The commands that have been issued so far have created a configuration document for each server.
These configuration documents are currently local to the Development OMAG Server Platform where the
adminstration commands were issued (figure 6).

![Figure 6](images/creating-configuration-documents.png)
> **Figure 6:** Creating configuration documents using administration commands

If servers are to be started on the other server platforms then their configuration documents
need to be deployed (copied) to these platforms (figure 7).

![Figure 7](images/deploying-configuration-documents.png)
> **Figure 7:** Deploying configuration documents

However, before deploying the configuration documents, the receiving OMAG Server Platforms
need to be running.

The code below checks the Core and Data Lake OMAG Server Platforms are running.

In [ ]:
print("\nChecking OMAG Server Platform availability...")

checkServerPlatform("Data Lake Platform", dataLakePlatformURL)
checkServerPlatform("Core Platform", corePlatformURL)
checkServerPlatform("Dev Platform", devPlatformURL)

print ("\nDone.")

----
Make sure the each of the platforms is running.

----
The commands below deploy the server configuration documents to the server platforms where the
servers will run.

In [ ]:


print("\nDeploying server configuration documents to appropriate platforms...")
    
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS1", dataLakePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS2", corePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS3", corePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS4", dataLakePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS5", corePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS6", corePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDSx", devPlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "exchangeDL01", dataLakePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "monitorDev01", devPlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "governDL01", dataLakePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoView1", dataLakePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoOLS1", dataLakePlatformURL)

print("\nDone.")

----